# Decision Trees using `scikit-learn`, `Pandas` and `NumPy`

In this code we'll attempt to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains 1151 instances and 20 attributes (with both continuous and categorical data). Dataset info [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

In [13]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import time

In [14]:
%matplotlib inline

In [15]:
# Read the data from csv file
col_names = []
for i in range(20):
    if i == 0:
        col_names.append('quality')
    if i == 1:
        col_names.append('prescreen')
    if i >= 2 and i <= 7:
        col_names.append('ma' + str(i))
    if i >= 8 and i <= 15:
        col_names.append('exudate' + str(i))
    if i == 16:
        col_names.append('euDist')
    if i == 17:
        col_names.append('diameter')
    if i == 18:
        col_names.append('amfm_class')
    if i == 19:
        col_names.append('label')

data = pd.read_csv("messidor_features.txt", names = col_names)
print(data.shape)
data.head(10)

(1151, 20)


,quality,prescreen,ma2,ma3,ma4,ma5,ma6,ma7,exudate8,exudate9,exudate10,exudate11,exudate12,exudate13,exudate14,exudate15,euDist,diameter,amfm_class,label
0,1,1,22,22,22,19,18,14,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,1,1,24,24,22,18,16,13,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,1,1,62,60,59,54,47,33,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,1,1,55,53,53,50,43,31,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,1,1,44,44,44,41,39,27,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1
5,1,1,44,43,41,41,37,29,28.356400,6.935636,2.305771,0.323724,0.000000,0.000000,0.000000,0.000000,0.502831,0.126741,0,1
6,1,0,29,29,29,27,25,16,15.448398,9.113819,1.633493,0.000000,0.000000,0.000000,0.000000,0.000000,0.541743,0.139575,0,1
7,1,1,6,6,6,6,2,1,20.679649,9.497786,1.223660,0.150382,0.000000,0.000000,0.000000,0.000000,0.576318,0.071071,1,0
8,1,1,22,21,18,15,13,10,66.691933,23.545543,6.151117,0.496372,0.000000,0.000000,0.000000,0.000000,0.500073,0.116793,0,1
9,1,1,79,75,73,71,64,47,22.141784,10.054384,0.874633,0.099780,0.023386,0.000000,0.000000,0.000000,0.560959,0.109134,0,1


### 1. Data preprocessing  & dimensionality reduction with PCA

The first block of code below will split the labels of the data into their own data structure (in this case a series/1D array). Next the second block will "standardize" the data such that the features are transformed to have a mean of 0 and a variance of 1. We are doing this as they will be required for PCA later.

In [16]:
classLabelCol=data.pop("label")

In [18]:
# StandardScaler "standardizes" the data and returns a NumPy array of the now standardized records
scaler = StandardScaler()
scaledFeatures = (scaler.fit_transform(data))
scaledFeatures

array([[ 0.05905386,  0.2982129 , -0.6414863 , ..., -1.29476283,
        -0.46865568,  1.40504812],
       [ 0.05905386,  0.2982129 , -0.56339113, ..., -0.08216786,
         2.00605415, -0.7117194 ],
       [ 0.05905386,  0.2982129 ,  0.92041699, ...,  0.27428264,
         1.1215164 , -0.7117194 ],
       ...,
       [ 0.05905386, -3.35330894,  0.41279842, ...,  1.33436273,
         1.19371332, -0.7117194 ],
       [ 0.05905386,  0.2982129 ,  0.0223226 , ..., -1.32796165,
        -0.09707846,  1.40504812],
       [ 0.05905386,  0.2982129 , -1.22720003, ...,  1.17603538,
        -1.08570243, -0.7117194 ]])

The code below uses sklearn's `train_test_split` object to split the data into an 80/20 train/test split so we can evaluate the model performance.

In [28]:
''' Data from train_test_split is returned as a 2D array with the following columns: training set of features,
 testing set of features, training set of labels, testing set of labels  
'''
splitData=train_test_split(scaledFeatures,classLabelCol, test_size=0.2, train_size=0.8)
print("Size of training data set: "+str(len(splitData[0])))
print("Size of test data set: "+str(len(splitData[1])))

Size of training data set: 920
Size of test data set: 231


We will be performing PCA in order to reduce the dimensionality of the dataset below. The `pca.explained_variance_ratio` attribute will indicate the percentage of the 95% variance we want to retain is explained by the new dimensional features after the transformation. We will transform the test set into the same dimensional space afterwards.

In [29]:
# The line below retains 95% variance for the PCA
pca = PCA(n_components=0.95)
pca.fit(splitData[0])
print("The below array shows the variance explained by the new post-PCA columns:\n")
print(pca.explained_variance_ratio_)
reducedTrainingData=pca.transform(splitData[0])
reducedTestingData=pca.transform(splitData[1])

The below array shows the variance explained by the new post-PCA columns:

[0.31875004 0.26623605 0.10509821 0.06409977 0.05785473 0.05079205
 0.03996146 0.03755909 0.0279813 ]


### 2. Training Decision Trees in `scikit-learn`

We will use `DecisionTreeClassifier` with as our split criterion being entropy (default is gini). We will use our newly transformed data that was outputted after using PCA.

In [30]:
decTree = DecisionTreeClassifier(criterion="entropy")
# Remember that splitData[2] gave us the labels for the training set
decTreeEstimator=decTree.fit(reducedTrainingData,splitData[2])

Now that we have trained a basic decision tree on a basic train section from a train/test split, we will go ahead and attempt to classify the test data and compare our predictions to the actual labels from that test set.

In [34]:
estimatedTestClasses=decTree.predict(reducedTestingData)
classifierAccuracy=accuracy_score(splitData[3],estimatedTestClasses)
print("Accuracy of classifier on test data: "+str(classifierAccuracy))

Accuracy of classifier on test data: 0.5757575757575758


Because of the numerous hyperparameters used in the decision tree classifier model, we can play around with some values to see how they affect accuracy.

In [38]:
''' As one can see in the accuracy score below, a few changes can affect the accuracy measure by 5% with some very 
minor tweaks to hyperparameters. Note that this does not mean the model is necessarily well generalizable!
'''
newClassifier = DecisionTreeClassifier(criterion = "gini", max_depth=3, min_samples_leaf=3)
newClassifier.fit(reducedTrainingData, splitData[2])
print(accuracy_score(splitData[3], newClassifier.predict(reducedTestingData)))


0.6233766233766234


### 3. Using K-fold Cross Validation

The 'holdout' method as mentioned previously is not enough to measure the generalized accuracy very well. We will use 'cross validation' in order to do so in a more robust way.

We will use `sklearn.model_selection.cross_val_score` to perform 10-fold cross validation on our decision tree and report the accuracy. Note the usage of the standardized pre-PCA transformation data. We will use PCA with k-fold cross validation later in another notebook.

In [40]:
oldDataTree = DecisionTreeClassifier(criterion="entropy")
oldDataCrossVal = cross_val_score(oldDataTree,scaledFeatures,classLabelCol,cv=10)
# The below line will average the accuracy from all the folds in order to get a better idea of accuracy
oldDataCrossVal.mean()

0.6229610194902547

We want to tune our model's hyperparameters in order to minimize potential overfitting of the model on our test data. To do so we will use the `GridSearchCV` object provided by sklearn. It will calculate the hyperparameter permutation with the best generalizable accuracy from our given data.

In [46]:
# Hyperparameters to test are give in the dict below
unknownValsDict = {"max_depth":range(5,21), "max_features":range(5,16), "min_samples_leaf":range(5,21)}
gridS = GridSearchCV(oldDataTree, unknownValsDict, scoring = 'accuracy', cv=5)
gridS.fit(scaledFeatures,classLabelCol)
print(gridS.best_params_)

{'max_depth': 7, 'max_features': 14, 'min_samples_leaf': 16}


In [47]:
# new code block to test accuracy of new hyperparameters
bestTree = DecisionTreeClassifier(criterion="entropy", max_depth=7, max_features=14, min_samples_leaf=16)
newCrossVal = cross_val_score(bestTree,scaledFeatures,classLabelCol,cv=5)
newCrossVal.mean()

0.6194541690193864

The code below will initiate a nested cross validation loop.

What this does is: the `cross_val_score` splits the data in to train and test sets for the first fold, and it passes the train set into `GridSearchCV`. `GridSearchCV` then splits that set into train and validation sets for k number of folds (the inner CV loop). The hyper-parameters for which the average score over all inner iterations is best, is reported as the `best_params_`, `best_score_`, and `best_estimator_`(best decision tree). This best decision tree is then evaluated with the test set from the `cross_val_score` (the outer CV loop). And this whole thing is repeated for the remaining k folds of the `cross_val_score` (the outer CV loop). 

We will print out the final accuracy of the model at the end.

In [48]:
# your code goes here
finalCrossVal = cross_val_score(gridS, scaledFeatures, classLabelCol, cv=5)
finalCrossVal.mean()

0.6255674760022585

Our accuracy rate isn't very good. We wouldn't want to use this model in the real world to actually diagnose patients due to it onl being accurate ~60% of the time. To improve this accuracy, other algorithms like boosting or bootstrapping could be useful. 